## Get the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.simplefilter('ignore')

## Cross validation: k-fold CV

The purpose of testing is to estimate a models quality of predicting data out of sample. A single train-test split of the data bears the risk of not being representative for a model's ability to generalize. Hence, where possible, multiple splits are preferred. An unordered dataset is typically split into folds. <u>Out of k folds each one is used as test set in turn</u>. There are as many train-test splits and scores as there are folds.

This maximizes the amount of data that is used to train the model, as during the course of training, the model is not only trained, but also tested on all of the available data. However, <u>the number of folds also determines the computational cost</u>.

In [2]:
from sklearn.model_selection import cross_val_score

boston_X, boston_y = datasets.load_boston(return_X_y=True)

reg = LinearRegression()
cv_results = cross_val_score(reg, boston_X, boston_y, cv=5)  # gives array of R2s
print(cv_results)

[ 0.63919994  0.71386698  0.58702344  0.07923081 -0.25294154]


## Hyperparameter tuning

### Basics

Hyperparameters are parameters that cannot be directley learned by fitting a model. Hyperparameter tuning chooses these parameters based on the success of the related model.

In scikit-learn hyperparameters are passed as arguments to the constructor of the estimator class. Examples include `C`, `kernel` and `gamma` for Support Vector Classifier and `alpha` (coefficient penalty) for Lasso.

One can search the hyper-parameter space for the best cross validation score. Any parameter provided when constructing an estimator may be optimized in this manner. To find names and values of all parameters for a given estimator one can use `estimator.get_params()`.

In [3]:
knn = KNeighborsClassifier()
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

### Grid search

A basic method for hyperparameter optimization is grid search. A <u>grid refers to combinations of plausible hyperparameter values</u>. The combination is then determined through a 'grid search'. The `sklearn` documentation shows the names of each model's hyperparameters.

**Example 1**: Find in-sample optimal number of neighbors for knn classifier with `GridSearchCV`.

In [4]:
X, y = datasets.load_iris(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': np.arange(1, 50)}  # grid is specified as dictionary of key-range pairs
knn = KNeighborsClassifier()  # initiate estimator

knn_cv = GridSearchCV(knn, param_grid, cv=5)  # initiate grid search object
knn_cv.fit(X_train, y_train)  # fit performs the actual grid search in place
print(knn_cv.best_params_)  # return the most successful hyperparameters
print(knn_cv.best_score_)  # return the score (here accuracy) of the most successful hyperparameters

{'n_neighbors': 3}
0.9714285714285715


**Example 2**: Find historically optimal regularization parameter for logistic regression

In [5]:
X, y = datasets.load_iris(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

from sklearn.model_selection import GridSearchCV

param_grid = {'C': np.logspace(-5, 8, 15)}  # Setup the hyperparameter grid
logreg = LogisticRegression()  # instantiate logistic regression classifier

logreg_cv = GridSearchCV(logreg, param_grid, cv=5)  # instantiate the GridSearchCV object
logreg_cv.fit(X, y)  # fit it to the data
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 31.622776601683793}
Best score is 0.9800000000000001


Hyperparameter tuning should split out a test or **hold-out set** before the tuning. The hold-out set serves as basis for testing the predictive quality of the overall method we used, i.e. hypterparameter tuning and estimation.

In [6]:
X, y = datasets.load_iris(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

from sklearn.model_selection import GridSearchCV

param_grid = {'C': np.logspace(-5, 8, 15), 'penalty': ['l1', 'l2']}  # Create the hyperparameter grid
logreg = LogisticRegression()  # instantiate the logistic regression classifier

logreg_cv = GridSearchCV(logreg, param_grid, cv=5)  # instantiate GridSearchCV object
logreg_cv.fit(X_train, y_train)  # fit it to the training data to hyperparameter tuning

print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {:.2f}".format(logreg_cv.best_score_))

y_pred = logreg_cv.predict(X_test)  # predict for the holdout test set
logreg_cv.score(X_test, y_test)  # checkout test accuracy
print("Test set accuracy: {:.2f}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameter: {'C': 3.727593720314938, 'penalty': 'l2'}
Tuned Logistic Regression Accuracy: 0.97
Test set accuracy: 0.97


### Randomized search

GridSearchCV can be computationally too expensive when searching over a large multi-variate hyperparameter spaces. Randomized search with `RandomizedSearchCV` is a "cheaper" solution in which not all hyperparameter values are tried out. Instead, a fixed number of hyperparameter settings is sampled from specified probability distributions. 

For example, decision trees have many parameters that can be tuned, such as `max_features`, `max_depth`, and `min_samples_leaf`. This makes it an ideal use case for randomized search.

In [7]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_dist = {"max_depth": [3, None],  # set up the parameters and distributions to sample from
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

tree = DecisionTreeClassifier()  # instantiate a Decision Tree classifier
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)  # instantiate the RandomizedSearchCV object
tree_cv.fit(X, y)  # fit the RSCV object to the data

print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Tuned Decision Tree Parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': 3, 'min_samples_leaf': 2}
Best score is 0.96


## Tryouts

`GridSearch` typically uses full pipelines and grids of hyperparameters to perform cross validation. It is itself a model that can be fit that supports optimization of hyperparameters. 

In [22]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

steps = [('scale', StandardScaler()), ('model', KNeighborsRegressor())]
pipe = Pipeline(steps)
print(pipe.get_params()) # view all settings of pipeline

{'memory': None, 'steps': [('scale', StandardScaler()), ('model', KNeighborsRegressor())], 'verbose': False, 'scale': StandardScaler(), 'model': KNeighborsRegressor(), 'scale__copy': True, 'scale__with_mean': True, 'scale__with_std': True, 'model__algorithm': 'auto', 'model__leaf_size': 30, 'model__metric': 'minkowski', 'model__metric_params': None, 'model__n_jobs': None, 'model__n_neighbors': 5, 'model__p': 2, 'model__weights': 'uniform'}


In [23]:
param_grid = {'model__n_neighbors': [i for i in range(1, 11)]} 
mod = GridSearchCV(estimator = pipe,  # the etimator is the whole pipeline
                  param_grid = param_grid,
                  cv = 3)
X, y = datasets.load_boston(return_X_y=True)
mod.fit(X, y)
df = pd.DataFrame(mod.cv_results_)  # pass output dictionary to pandas df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mean_fit_time             10 non-null     float64
 1   std_fit_time              10 non-null     float64
 2   mean_score_time           10 non-null     float64
 3   std_score_time            10 non-null     float64
 4   param_model__n_neighbors  10 non-null     object 
 5   params                    10 non-null     object 
 6   split0_test_score         10 non-null     float64
 7   split1_test_score         10 non-null     float64
 8   split2_test_score         10 non-null     float64
 9   mean_test_score           10 non-null     float64
 10  std_test_score            10 non-null     float64
 11  rank_test_score           10 non-null     int32  
dtypes: float64(9), int32(1), object(2)
memory usage: 1.0+ KB


In [21]:
df.set_index('rank_test_score')[['param_model__n_neighbors', 'mean_test_score', 'std_test_score']].sort_index()

,param_model__n_neighbors,mean_test_score,std_test_score
rank_test_score,,,
1,3,0.402900,0.064986
2,8,0.398003,0.155124
3,9,0.394387,0.165643
4,10,0.391870,0.184741
5,7,0.391216,0.123003
6,6,0.390532,0.116406
7,4,0.383619,0.083675
8,5,0.374286,0.103638
9,2,0.313246,0.101821


In [9]:
[i for i in range(1, 11)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

https://www.youtube.com/watch?v=0B5eIE_1vpU

https://www.youtube.com/watch?v=4PXAztQtoTg

https://scikit-learn.org/stable/tutorial/index.html#tutorial-menu

now:
https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html